In [1]:
import numpy as np
import time
import requests
from bs4 import BeautifulSoup
from z3 import *

In [2]:
# More z3 magic

url='https://www.janestreet.com/puzzles/wrong-division/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
print("Puzzle")
print("~~~~~~")
print(" ".join(y[7:8]))

Puzzle
~~~~~~
What We Do


<img src="https://www.janestreet.com/puzzles/Nov14.png" width="300" >

In [3]:
def to_number(x):
    numb = 0
    for i in range(len(x)):
        numb += (10**i) *x[-(i+1)] 
    return numb     

In [4]:
start = time.time()

#set up the variables and the numbers
divisor  = [Int("divisor_{}".format(i)) for i in range(4)] 
dividend = [Int("dividend_{}".format(i)) for i in range(8)]
quotient = [Int("quotient_{}".format(i)) for i in range(5)] 
row1 = [Int("row1_{}".format(i)) for i in range(4)] 
row2 = [Int("row2_{}".format(i)) for i in range(5)] 
row3 = [Int("row3_{}".format(i)) for i in range(5)] 
row4 = [Int("row4_{}".format(i)) for i in range(4)] 
row5 = [Int("row5_{}".format(i)) for i in range(4)] 
row6 = [Int("row6_{}".format(i)) for i in range(5)] 

s = Tactic("pqffd").solver()

#numbers in a range
s += [And(0 <= divisor[i],divisor[i] <= 9) for i in range(4)]
s += [And(0 <= dividend[i],dividend[i] <= 9) for i in range(8)]
s += [And(0 <= quotient[i],quotient[i] <= 9) for i in range(5)]
s += [And(0 <= row1[i],row1[i] <= 9) for i in range(4)]
s += [And(0 <= row2[i],row2[i] <= 9) for i in range(5)]
s += [And(0 <= row3[i],row3[i] <= 9) for i in range(5)]
s += [And(0 <= row4[i],row4[i] <= 9) for i in range(4)]
s += [And(0 <= row5[i],row5[i] <= 9) for i in range(4)]
s += [And(0 <= row6[i],row6[i] <= 9) for i in range(5)]

# the long division
# ~~~~~~~~~~~~~~~~~
divisor_num  = sum([divisor[-(i+1)] * (10**i) for i in range(4)])
dividend_num  = sum([dividend[-(i+1)] * (10**i) for i in range(8)])
quotient_num  = sum([quotient[-(i+1)] * (10**i) for i in range(5)])

s +=  dividend_num == divisor_num * quotient_num

# line by line
# ~~~~~~~~~~~~

# the first quotient
initial_num = sum([dividend[:4][-(i+1)] * (10**i) for i in range(4)])
row1_num = sum([row1[-(i+1)] * (10**i) for i in range(4)])
remainder1 = sum([row2[:3][-(i+1)] * (10**i) for i in range(3)])

s += row1_num == divisor_num * quotient[0]
s += initial_num - row1_num == remainder1

# the second quotient

s += quotient[1] == 0

# the third quotient
row2_num = sum([row2[-(i+1)] * (10**i) for i in range(5)])
row3_num = sum([row3[-(i+1)] * (10**i) for i in range(5)])
remainder2 = sum([row4[:3][-(i+1)] * (10**i) for i in range(3)])

s += row2_num == remainder1*100+dividend[4]*10+dividend[5] 
s += row3_num == divisor_num * quotient[2]
s += row2_num - row3_num == remainder2

# the fourth quotient
row4_num = sum([row4[-(i+1)] * (10**i) for i in range(4)])
row5_num = sum([row5[-(i+1)] * (10**i) for i in range(4)])
remainder3 = sum([row6[:4][-(i+1)] * (10**i) for i in range(4)])

s += row4_num == remainder2*10+dividend[6] 
s += row5_num == divisor_num * quotient[3]
s += row4_num - row5_num == remainder3

# the fifth quotient
row6_num = sum([row6[-(i+1)] * (10**i) for i in range(5)])
s += row6_num == remainder3*10+dividend[7] 
s += row6_num == divisor_num * quotient[4]

# Fixed numbers
# ~~~~~~~~~~~~~
s += Or(row1[1]==4,row1[1]==6)
s += Or(row1[3]==4,row1[3]==6)

s += row2[0]==2
s += Or(row2[2]==1,row2[2]==3)
s += Or(row2[4]==4,row2[4]==6)

s += row3[0]==1
s += Or(row3[2]==7,row3[2]==9)

s += Or(row4[0]==4,row3[0]==6)
s += Or(row4[2]==7,row3[2]==9)
s += Or(row4[3]==0,row3[3]==2)

s += Or(row5[0]==2,row5[0]==4)
s += Or(row5[1]==6,row5[1]==8)

s += Or(row6[0]==1,row6[0]==3)
s += Or(row6[3]==1,row6[3]==3)

#can't be a leading zero
s += dividend[0]>0
s += quotient[0]>0
s += divisor[0]>0

###################
# Solve and Print #
###################

if s.check() == sat:
    m = s.model()
    
    divisor_result =  to_number([ m.evaluate(divisor[i]).as_long() for i in range(4) ] )
    dividend_result = to_number([ m.evaluate(dividend[i]).as_long() for i in range(8) ] )
    quotient_result = to_number([ m.evaluate(quotient[i]).as_long() for i in range(5) ] )
    row1_result = to_number([ m.evaluate(row1[i]).as_long() for i in range(4) ] )
    row2_result = to_number([ m.evaluate(row2[i]).as_long() for i in range(5) ] )
    row3_result = to_number([ m.evaluate(row3[i]).as_long() for i in range(5) ] )
    row4_result = to_number([ m.evaluate(row4[i]).as_long() for i in range(4) ] )
    row5_result = to_number([ m.evaluate(row5[i]).as_long() for i in range(4) ] )
    row6_result = to_number([ m.evaluate(row6[i]).as_long() for i in range(5) ] )
    print("Solved in {:.4} seconds".format(time.time()-start))
    print("\n{:,.0f} / {:,.0f} = {:,.0f}\n".format(dividend_result,divisor_result,quotient_result))
    print("Check :",dividend_result/ divisor_result == quotient_result)  # sometimes gets stuck very close to the correct solution
    print("row1: {:,.0f}".format(row1_result))
    print("row2: {:,.0f}".format(row2_result))
    print("row3: {:,.0f}".format(row3_result))
    print("row4: {:,.0f}".format(row4_result))
    print("row5: {:,.0f}".format(row5_result))
    print("row6: {:,.0f}".format(row6_result))
                    
else:
    print("Failed")

Solved in 0.6403 seconds

58,975,605 / 2,847 = 20,715

Check : True
row1: 5,694
row2: 20,356
row3: 19,929
row4: 4,270
row5: 2,847
row6: 14,235


In [5]:
url='https://www.janestreet.com/puzzles/solutions/november-2014-solution/'
res = requests.get(url)
soup = BeautifulSoup(res.content, 'html.parser')
y =[text for text in soup.body.stripped_strings]
print("Solution")
print("~~~~~~~~")
print(" ".join(y[7:8]))

Solution
~~~~~~~~
What We Do


<img src="https://www.janestreet.com/puzzles/Nov14_solution.png" width="300" >

In [6]:
s.statistics()

(:max-memory              16.74
 :memory                  3.52
 :num-allocs              127517982
 :par-progress            100.00
 :rlimit-count            2300857
 :sat-backjumps           7297
 :sat-conflicts           7297
 :sat-decisions           14704
 :sat-del-clause          44837
 :sat-elim-bool-vars-res  4554
 :sat-elim-clauses        15214
 :sat-elim-literals       10451
 :sat-minimized-lits      67811
 :sat-mk-clause-2ary      23709
 :sat-mk-clause-3ary      38405
 :sat-mk-clause-nary      29186
 :sat-mk-var              17522
 :sat-probing-assigned    2135
 :sat-propagations-2ary   1038695
 :sat-propagations-3ary   539567
 :sat-propagations-nary   427681
 :sat-restarts            508
 :sat-scc-elim-binary     42
 :sat-scc-elim-vars       1752
 :sat-subs-resolution     111
 :sat-subs-resolution-dyn 3544
 :sat-subsumed            3172
 :sat-units               6718
 :time                    0.61)